In [1]:
!pip install --upgrade openai


In [2]:
import os
import openai
from getpass import getpass

# Enter your API key securely (won't display)
os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = os.environ["OPENAI_API_KEY"]

🔑 Enter your OpenAI API key: ··········


In [3]:
response = openai.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Summarize the pros and cons of aspirin."}
    ],
    temperature=0.5,
    max_tokens=300
)

print(response.choices[0].message.content)

Aspirin, one of the oldest and most commonly used medications worldwide, has a variety of uses primarily in pain relief, anti-inflammatory treatment, and cardiovascular protection. Here's a summary of its pros and cons:

**Pros of Aspirin:**
1. **Pain Relief:** Aspirin is effective in reducing pain from various conditions such as headaches, muscle aches, and toothaches.
2. **Anti-inflammatory:** It helps reduce inflammation, making it beneficial for treating conditions like rheumatoid arthritis.
3. **Antipyretic Effects:** Aspirin is useful in lowering fever.
4. **Prevention of Blood Clots:** It inhibits platelet aggregation, thereby reducing the risk of blood clots. This is particularly beneficial in preventing strokes and heart attacks in appropriate populations.
5. **Cardiovascular Benefits:** Regular use of low-dose aspirin can prevent the recurrence of heart attacks and strokes in individuals with a history of these conditions.
6. **Possible Cancer Prevention:** Some studies sugge

In [4]:
# Scenario-specific clinical data
scenarios = [
    {
        "name": "The Triage Bottleneck", "role": "triage",
        "age": "71", "sex": "Male",
        "symptoms": "Shortness of breath, chest tightness (HR 108, SpO₂ 92 %, BP 165/95)",
        "history": "Chronic sinusitis, essential hypertension, hyperlipidaemia",
        "meds": "None recorded",
        "context": "Walk-in, 20:00 Friday, Evangelismós General Hospital, Athens"
    },
    {
        "name": "The Alert That Was Ignored", "role": "alert",
        "age": "55", "sex": "Female",
        "symptoms": "Acute bronchitis, wheezing",
        "history": "Severe asthma, hypertension, prediabetes",
        "meds": "Fluticasone, Albuterol, Hydrochlorothiazide",
        "context": "Inpatient at 02:15, night shift, Ippokráteio Hospital, Thessaloníki"
    },
    {
        "name": "The Rural GP and the Black Box", "role": "gp_diagnosis",
        "age": "45", "sex": "Male",
        "symptoms": "Vague sore throat, low-grade fever",
        "history": "No chronic conditions recorded",
        "meds": "None",
        "context": "GP visit, Agiásos village clinic, island of Lésvos"
    },
    {
        "name": "Second Opinion or Surveillance", "role": "second_opinion",
        "age": "51", "sex": "Female",
        "symptoms": "Breast carcinoma with lung metastases",
        "history": "Chronic sinusitis, cystitis, prediabetes, prior tubal pregnancy",
        "meds": "Mirena IUD, Chlorpheniramine, Errin",
        "context": "Oncology MDT, Laiko University Hospital, Athens"
    },
    {
        "name": "The Data Privacy Trap", "role": "risk_scorer",
        "age": "29", "sex": "Male",
        "symptoms": "Type-2 diabetes follow-up, possible non-adherence",
        "history": "Major depressive disorder, nicotine dependence (inferred)",
        "meds": "Acetaminophen, antidepressants",
        "context": "Outpatient diabetes clinic, General Hospital of Pátras"
    }
]

# Role-specific system prompts
system_prompts = {
    "triage": (
        "You are a clinical AI decision-support assistant embedded in the emergency department "
        "of a public hospital in Greece. You recommend urgency level and department based on presentation. "
        "Your advice is advisory. Always communicate uncertainty."
    ),
    "alert": (
        "You are an AI medication-alert system in a hospital EHR. You detect adverse drug events, "
        "trigger alerts, and escalate risk while avoiding alert fatigue."
    ),
    "gp_diagnosis": (
        "You are an AI diagnostic assistant in a rural GP’s clinic. You suggest likely diagnoses "
        "and flag red-flag symptoms. Support, not replace, clinical judgment."
    ),
    "second_opinion": (
        "You are a patient-facing AI for second opinions based on medical notes and literature. "
        "Your output is visible to clinicians and must be explainable and caveated."
    ),
    "risk_scorer": (
        "You are a risk-scoring AI module in a public hospital dashboard. You infer risk of complications "
        "using structured and inferred data. Flag when data is privacy-sensitive or inferred."
    )
}

In [5]:
# Function to build the user prompt from each scenario
def build_user_prompt(entry):
    return (
        f"Setting\n{entry['context']}\n\n"
        "Patient Profile\n"
        f"- Age: {entry['age']}\n"
        f"- Sex: {entry['sex']}\n"
        f"- Presenting Symptoms: {entry['symptoms']}\n"
        f"- Medical History: {entry['history']}\n"
        f"- Current Medications: {entry['meds']}\n"
        f"- Encounter Context: {entry['context']}\n\n"
        "Task:\n"
        "1. Summarise the patient's condition and identify signs of risk.\n"
        "2. Recommend an appropriate action (triage level, alert, diagnosis, risk score, etc.).\n"
        "3. Justify your decision using the clinical data provided.\n"
        "4. Note any limitations or uncertainty in your response.\n"
        "5. Flag any data elements that require clinician verification."
    )


In [6]:
import datetime
import csv

# Store results for all scenarios
results = []

# Run each scenario through GPT-4-turbo
for entry in scenarios:
    print(f"⏳ Querying GPT-4-turbo for: {entry['name']}")

    response = openai.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": system_prompts[entry["role"]]},
            {"role": "user", "content": build_user_prompt(entry)}
        ],
        temperature=0.4,
        max_tokens=700
    )

    # Extract result data
    content = response.choices[0].message.content.strip()
    timestamp = datetime.datetime.utcnow().isoformat(timespec="seconds")
    usage = response.usage

    results.append({
        "scenario": entry["name"],
        "timestamp_utc": timestamp,
        "prompt_tokens": usage.prompt_tokens,
        "completion_tokens": usage.completion_tokens,
        "total_tokens": usage.total_tokens,
        "model": "gpt-4-turbo",
        "response": content
    })

print("\n✅ All queries complete.")


⏳ Querying GPT-4-turbo for: The Triage Bottleneck
⏳ Querying GPT-4-turbo for: The Alert That Was Ignored
⏳ Querying GPT-4-turbo for: The Rural GP and the Black Box
⏳ Querying GPT-4-turbo for: Second Opinion or Surveillance
⏳ Querying GPT-4-turbo for: The Data Privacy Trap

✅ All queries complete.


In [7]:
# Save results to CSV
csv_path = "llm_outputs.csv"
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=results[0].keys())
    writer.writeheader()
    writer.writerows(results)

print(f"✅ Results saved to: {csv_path}")


✅ Results saved to: llm_outputs.csv
